### Ejecting from a moving object

By default, `speedystar` ejects stars from the Galactic Centre. The ejection methods in `speedystar.eject` can take a `galpy` orbit object as an argument to eject stars from a moving object (e.g. the LMC, M31, globular clusters), though the moving object must be set up in a particular way to ensure stars are ejected from the correct location at the correct time. Here we will use the LMC as an example.

In [ ]:
import numpy as np
from speedystar import starsample
from speedystar.eject import Hills, BMBH
import astropy.units as u
from galpy.potential.mwpotentials import MWPotential2014
from galpy.potential import MovingObjectPotential, HernquistPotential
from galpy import potential
from galpy.orbit import Orbit
from astropy.table import Table

In [2]:
# Assume a Galactic potential
default_potential = MWPotential2014

#Ensure the potential has physical units so that the final positions and velocities have physical units too
potential.turn_physical_on(default_potential)

#Initialize LMC orbit object using contemporary estimates of its centre position, distance and velocity
#Note: If the optional arguments ro, vo, zo, solarmotion are changed here, make sure they are changed in the mysample.propagate() call below as well, and vice versa. May result in inconsistent behaviour otherwise. 
LMCorbit = Orbit(vxvv = [78.76*u.deg, -69.19*u.deg, 49.59*u.kpc, 1.91*u.mas/u.yr, 0.229*u.mas/u.yr, 262.2*u.km/u.s], radec=True)

#Note: A realistic treatment of the LMC orbit should include a ChandrasekharDynamicalFrictionForce() component of the potential as well to account for dynamical friction. Not included here.

#Set integration times for the object.
#Note the setup. The first integration timestep MUST be zero and the timesteps must be decreasing.
#The reason the orbit must be integrated like this instead of, say, flipping the orbit with galpy.orbit.Orbit.flip() and integrating towards positive times is that speedystar.eject() and speedystar.propagate() use the object's galpy.orbit.Orbit.time attribute to determine where to eject stars from and through what potential they move through, and both expect this attribute to be strictly non-positive and to be located at its initialized position and velocity at t=0.
ts = np.linspace(0, -2000, 2000)*u.Myr
LMCorbit.integrate(ts,default_potential)

#Uncomment these to save the moving object orbit to file. Useful for debugging and sanity checking.

#ra, dec, dist = LMCorbit.ra(ts,quantity=True), LMCorbit.dec(ts,quantity=True), LMCorbit.dist(ts, quantity=True)
#pmra, pmdec, vlos = LMCorbit.pmra(ts,quantity=True), LMCorbit.pmdec(ts,quantity=True), LMCorbit.vlos(ts, quantity=True)
#x, y, z = LMCorbit.x(ts,quantity=True), LMCorbit.y(ts,quantity=True), LMCorbit.z(ts, quantity=True)
#vx, vy, vz = LMCorbit.vx(ts,quantity=True), LMCorbit.vy(ts,quantity=True), #LMCorbit.vz(ts, quantity=True)

#namelist =['t', 'ra', 'dec', 'dist', 'pmra', 'pmdec', 'vlos', 'x', 'y', 'z', 'vx', 'vy', 'vz']
#datalist = [ts, ra, dec, dist, pmra, pmdec, vlos, x, y, z, vx, vy, vz]
#data_table = Table(data=datalist, names=namelist)
#data_table.write('./LMC_orbit.fits',overwrite=True)

ejectionmodel = Hills(rate=1e-4/u.yr, LaunchLoc=LMCorbit)

# Eject the sample
mysample = starsample(ejectionmodel, name='My catalogue')

# Save ejection sample
mysample.save('./cat_ejection_moving.fits')

#Note that the phase space coordinates of the stars in the sample are in the Galactocentric frame. To convert them to the LMC frame, we need to calculate the velocity of the LMC at the time of ejection of each star and subtract it from the star's velocity in the Galactocentric frame.

#Velocity of the LMC when each star was launched
LMCvx0, LMCvy0, LMCvz0 = LMCorbit.vx(-mysample.tflight,quantity=True), LMCorbit.vy(-mysample.tflight,quantity=True), LMCorbit.vz(-mysample.tflight, quantity=True)

#Components of the star's initial velocity in the Galactocentric frame
mysample.v0x = np.cos(mysample.phiv0)*np.sin(mysample.thetav0)*mysample.v0
mysample.v0y = np.sin(mysample.phiv0)*np.sin(mysample.thetav0)*mysample.v0
mysample.v0z = np.cos(mysample.thetav0)*mysample.v0

#Initial velocity in the LMC frame
mysample.v0_LMC = np.sqrt ( (mysample.v0x - LMCvx0)**2 + (mysample.v0y - LMCvy0)**2 + (mysample.v0z - LMCvz0)**2)


Evolving HVSs: 100%|██████████| 871/871 [00:02<00:00, 394.74it/s]


In [3]:
#Propagating the sample

# Load ejection sample, if it doesn't already exist
mysample = starsample('./cat_ejection_moving.fits')

#Realistically, densities around a moving ejection location are likely high enough to have a non-negligible contribution to the Galactic potential. Including a MovingObjectPotential component in the total potential is recommended, but not necessary. Here we will model the LMC as a Hernquist potential.

#LMC scale mass and size, as assumed in e.g. Erkal+2020 (MNRAS, 483, 2007)
Mlmc = 1.5e11*u.Msun
Rlmc = 17.14*u.kpc

#Create moving LMC potential. Hernquist potential amplitude is twice the mass in galpy.
LMCp = MovingObjectPotential(orbit=LMCorbit, pot=HernquistPotential(amp=2*Mlmc, a=Rlmc))

#Add moving LMC potential to total Galactic potential.
pottotal = [default_potential,LMCp]

#Make sure the potential is physical, just in case
potential.turn_physical_on(pottotal)

#Propagate the sample through the Milky Way + LMC potential. Note that propagation through any potential which involves a moving object is significantly slower than through a static potential.
#You may get a galpy warnings about integration time units. Ignore this.

mysample.propagate(potential = pottotal)

#Save propagated sample
mysample.save('./cat_propagatedLMC.fits')

Propagating...:   0%|          | 0/577 [00:00<?, ?it/s]

Propagating...: 100%|██████████| 577/577 [00:25<00:00, 22.56it/s]
